In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

In [ ]:
!pip install import_ipynb==0.1.3
!pip install chess

In [ ]:
import chess
import json

import import_ipynb
import Game
import Constants
from MinimaxAlgorithm import make_move_minimax, make_move_minimax_memoization
from RandomAlgorithm import make_random_move
from AlphaBetaAlgorithm import make_move_alphabeta, make_move_alphabeta_iterative_deepening




In [ ]:
!ls /datasets/gaviota

In genreal, a game of chess can be initialized through the instance of a Game-class. To be precise, the creation of the game-object can be "customized" with the following parameters:

    game = Game.Game(  
                make_move_algorithm_white = " desired_algo or None (human player) " ,  
                make_move_algorithm_black = " desired_algo or None (human player) " ,  
                opening_book              = " Path to opening_book-library or None"        ,  
                endgame_tablebase_dir     = " Path to endgame_tablebase or None"  
                )

in this case, the make_move_algorithm white and black represent the different algorithms which can play against each other or against a human enemy. In case a human wants to play against an algorithm, simply set 

    make_move_algorithm = None
    
as descriped above (Leaving out the argument has the same effect). 
The possible algorithms are:  

1. make_random_move
2. make_move_minimax
3. make_move_minimax_memoization
4. make_move_alphabeta
5. make_move_alphabeta_iterative_deepening  

Furthermore, it is recommended to use both the opening_book as well as the endgame_tablebase_dir argument as not None in order to boost efficiency and reduce the time needed by the AI to make a move.

Here is a basic game which can be played. The ai's are represented through the minimax-algorithm and the alphabeta-algorithm. In this case, the endgame-table is not implemented. 

In [ ]:
game = Game.Game(make_move_minimax, make_move_alphabeta, endgame_tablebase_dir= None)
#game.play()

___
This is for debbuging purposes. In the following cell we can see multiple tests executed in a loop. Here, all possible tests are run.


In [ ]:
chess_problems = Constants.chess_problems
for c in chess_problems:
    game.debug_play(c, automation = True)

In the following cell the test-details of all previews tests are displayed.

In [ ]:
import pprint #importing prettyprint for prettier print :D

pprint.pprint(game.auto_tests)
print(f'\n{game.wins}')

___

In the following case, a chess_problem is given in the function call. After that there is an example of a false argument

In [ ]:
game.debug_plööay(chess_problem = "EK")

In [ ]:
game.debug_play(chess_problem = "H")


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>